In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fastf1
from fastf1 import plotting
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib import rcParams

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025, 13, 'Qualifying')
session.load()

In [ ]:
pia_color = plotting.get_driver_color(session=session,identifier='PIA')
nor_color = 'yellow'

In [ ]:
nor_df = pd.DataFrame(session.laps.pick_drivers('NOR').pick_fastest().telemetry[['Time','Distance','X','Y','Speed','nGear','Throttle','Brake']]).copy()
pia_df = pd.DataFrame(session.laps.pick_drivers('PIA').pick_fastest().telemetry[['Time','Distance','X','Y','Speed','nGear','Throttle','Brake']]).copy()

In [ ]:
nor_df['RelTime'] = nor_df['Time'].dt.total_seconds()
nor_df.drop('Time',axis=1,inplace=True)
nor_df['Brake'] = nor_df['Brake'].apply(lambda x: 1 if x else 0)
pia_df['RelTime'] = pia_df['Time'].dt.total_seconds()
pia_df.drop('Time',axis=1,inplace=True)
pia_df['Brake'] = pia_df['Brake'].apply(lambda x: 1 if x else 0)

In [ ]:
# Create common distance base
common_distance = np.linspace(
    0,
    min(nor_df['Distance'].max(), pia_df['Distance'].max()),
    7000
)

In [ ]:
def interpolate_on_distance(df, common_distance):
    df = df.copy()

    # Prepare output
    interp_df = pd.DataFrame({'Distance': common_distance})

    # Interpolate each column
    for col in ['X', 'Y', 'Speed', 'RelTime', 'nGear', 'Throttle', 'Brake']:
        interp_df[col] = np.interp(
            common_distance,
            df['Distance'],
            pd.to_numeric(df[col])
        )

    return interp_df

In [ ]:
nor_interp_dist = interpolate_on_distance(nor_df,common_distance)
pia_interp_dist = interpolate_on_distance(pia_df,common_distance)

In [ ]:
nor_interp_dist['X'] = (nor_interp_dist['X'] + pia_interp_dist['X'])/2
pia_interp_dist['X'] = (nor_interp_dist['X'] + pia_interp_dist['X'])/2

nor_interp_dist['Y'] = (nor_interp_dist['Y'] + pia_interp_dist['Y'])/2
pia_interp_dist['Y'] = (nor_interp_dist['Y'] + pia_interp_dist['Y'])/2

In [ ]:
# Create common time base
common_time = np.linspace(
    0,
    min(nor_df['RelTime'].max(), pia_df['RelTime'].max()),
    7000
)

In [ ]:
def interpolate_on_time(df, common_time):
    df = df.copy()

    # Prepare output
    interp_df = pd.DataFrame({'RelTime': common_time})

    # Interpolate each column
    for col in ['X', 'Y', 'Speed', 'Distance', 'nGear', 'Throttle', 'Brake']:
        interp_df[col] = np.interp(
            common_time,
            df['RelTime'],
            pd.to_numeric(df[col])
        )

    return interp_df

In [ ]:
nor_interp_time = interpolate_on_time(nor_interp_dist,common_time)
pia_interp_time = interpolate_on_time(pia_interp_dist,common_time)

In [ ]:
def smooth_relative_to_anchor(anchor_df, target_df, window=5):
    """
    Smooth target_df's X and Y coordinates relative to anchor_df's position.
    
    Parameters:
        anchor_df: DataFrame with 'X', 'Y' columns (anchor car)
        target_df: DataFrame with 'X', 'Y' columns (car to smooth)
        window: Window size for rolling average
    
    Returns:
        target_smoothed_df: DataFrame with smoothed 'X', 'Y'
    """
    # Ensure alignment by index
    assert len(anchor_df) == len(target_df), "DataFrames must have same length"

    # Step 1: Compute deltas
    delta_x = target_df['X'] - anchor_df['X']
    delta_y = target_df['Y'] - anchor_df['Y']

    # Step 2: Smooth deltas
    delta_x_smooth = delta_x.rolling(window, center=True, min_periods=1).mean()
    delta_y_smooth = delta_y.rolling(window, center=True, min_periods=1).mean()

    # Step 3: Add back to anchor
    target_smoothed = target_df.copy()
    target_smoothed['X'] = anchor_df['X'] + delta_x_smooth
    target_smoothed['Y'] = anchor_df['Y'] + delta_y_smooth

    return target_smoothed

In [ ]:
nor_interp_time = smooth_relative_to_anchor(pia_interp_time,nor_interp_time,window=150)

In [ ]:
# Set animation size limit
rcParams['animation.embed_limit'] = 500

In [ ]:
# --- Set Up Plot ---
fig, ax = plt.subplots(figsize=(10, 8))
fig.set_facecolor('#333333')
ax.set_title('Qualifying Lap Comparison (VER vs PIA)', fontsize=14)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.axis('off')

# Base track
ax.plot(nor_interp_dist['X'], nor_interp_dist['Y'], 'k', linewidth=18, zorder=0)

# Moving markers and trails
nor_marker, = ax.plot([], [], marker='o', color=nor_color, ms=10, label='NOR')
pia_marker, = ax.plot([], [], marker='o', color=pia_color, ms=10, label='PIA')
nor_trail, = ax.plot([], [], color=nor_color, lw=1)
pia_trail, = ax.plot([], [], color=pia_color, lw=1)

# # Text box for metrics
# text_box = ax.text(
#     0.05, 0.95, '', transform=ax.transAxes,
#     fontsize=11, verticalalignment='top',
#     bbox=dict(facecolor='white', alpha=0.8)
# )

# --- Animation Function ---
def animate(i):
    nor_seg = nor_interp_time.iloc[:i+1]
    pia_seg = pia_interp_time.iloc[:i+1]

    # Marker positions
    nor_marker.set_data([nor_seg['X'].iloc[-1]], [nor_seg['Y'].iloc[-1]])
    pia_marker.set_data([pia_seg['X'].iloc[-1]], [pia_seg['Y'].iloc[-1]])

    # Get latest positions
    pia_x = pia_seg['X'].iloc[-1]
    pia_y = pia_seg['Y'].iloc[-1]

    # Compute center and margin
    center_x = pia_x
    center_y = pia_y

    # Dynamic margin (meters)
    margin = 2500  # Adjust based on your track scale

    # Set axis limits
    ax.set_xlim(center_x - margin, center_x + margin)
    ax.set_ylim(center_y - margin, center_y + margin)

    # Trails
    trail_len = 500  # or 100
    nor_trail.set_data(nor_seg['X'].iloc[-trail_len:], nor_seg['Y'].iloc[-trail_len:])
    pia_trail.set_data(pia_seg['X'].iloc[-trail_len:], pia_seg['Y'].iloc[-trail_len:])

    # # Distances
    # ver_seg_dist = ver_seg['Distance'].diff().fillna(0)
    # pia_seg_dist = pia_seg['Distance'].diff().fillna(0)

    # ver_total_dist = ver_seg['Distance'].iloc[-1]
    # pia_total_dist = pia_seg['Distance'].iloc[-1]

    # # % Full Throttle
    # ver_throttle_dist = ver_seg_dist[ver_seg['Throttle'] == 100].sum()
    # pia_throttle_dist = pia_seg_dist[pia_seg['Throttle'] == 100].sum()
    # ver_throttle_pct = (ver_throttle_dist / ver_total_dist) * 100 if ver_total_dist else 0
    # pia_throttle_pct = (pia_throttle_dist / pia_total_dist) * 100 if pia_total_dist else 0

    # # % Brake
    # ver_brake_dist = ver_seg_dist[ver_seg['Brake'] == True].sum()
    # pia_brake_dist = pia_seg_dist[pia_seg['Brake'] == True].sum()
    # ver_brake_pct = (ver_brake_dist / ver_total_dist) * 100 if ver_total_dist else 0
    # pia_brake_pct = (pia_brake_dist / pia_total_dist) * 100 if pia_total_dist else 0

    # # Gearshifts
    # ver_gearshifts = ver_seg['nGear'].diff().fillna(0).ne(0).sum()
    # pia_gearshifts = pia_seg['nGear'].diff().fillna(0).ne(0).sum()

    # # Speeds
    # ver_speed = ver_seg['Speed'].iloc[-1]
    # pia_speed = pia_seg['Speed'].iloc[-1]

    # # Update text
    # text_box.set_text(
    #     f"VER  | Throttle: {ver_throttle_pct:.1f}% | Brake: {ver_brake_pct:.1f}% | Gearshifts: {ver_gearshifts} | Speed: {ver_speed:.1f} km/h\n"
    #     f"PIA  | Throttle: {pia_throttle_pct:.1f}% | Brake: {pia_brake_pct:.1f}% | Gearshifts: {pia_gearshifts} | Speed: {pia_speed:.1f} km/h"
    # )

    return nor_marker, pia_marker, nor_trail, pia_trail #, text_box

# --- Run Animation ---
ani = FuncAnimation(fig, animate, frames=len(common_time), interval=20, blit=True, repeat=False)

# Optional: save
# ani.save("qualifying_comparison.mp4", fps=30, dpi=200)

plt.legend()
plt.tight_layout()
HTML(ani.to_jshtml())